# Test modules

Will test classes and functionalities of `sparql.py` and `pd_stardog_queries.py`.

In [1]:
# import the database connection class
from sparql import DB

In [2]:
import json

In [3]:
# config_file = "config.local.json"
config_file = "config.poetrylab.json"

In [4]:
with open(config_file) as f:
    config = json.load(f)

## Database connection `DB`
Test the Database connection with a simple query.

In [5]:
# create the connection using the DB class
db = DB(
    triplestore="stardog",
    protocol=config["server"]["protocol"],
    url=config["server"]["url"],
    port=config["server"]["port"],
    username=config["server"]["credentials"]["user"],
    password=config["server"]["credentials"]["password"],
    database=config["server"]["database"])


In [6]:
query = """
SELECT * WHERE {
    ?s ?p ?o .
}
LIMIT 1
"""

In [7]:
# send the test query to the database
db.sparql(query)

{'head': {'vars': ['s', 'p', 'o']},
 'results': {'bindings': [{'p': {'type': 'uri',
     'value': 'http://postdata.linhd.uned.es/ontology/postdata-poeticAnalysis#graphName'},
    's': {'type': 'uri',
     'value': 'http://postdata.linhd.uned.es/resource/sc_juana-ines-de-la-cruz_sabras-querido-fabio_plc_16454644863503196'},
    'o': {'type': 'uri',
     'value': 'http://postdata.linhd.uned.es/A_juana-ines-de-la-cruz_sabras-querido-fabio_16454644863503196'}}]}}

## class `SparqlQuery`

In [8]:
from sparql import SparqlQuery

In [9]:
# load the query from above
test = SparqlQuery(query=query, label="Test", description="Just get one single arbitrary triple.")

In [10]:
test

In [11]:
# output the query
test.query

'\nSELECT * WHERE {\n    ?s ?p ?o .\n}\nLIMIT 1\n'

In [12]:
# has the same effect
test.dump()

'\nSELECT * WHERE {\n    ?s ?p ?o .\n}\nLIMIT 1\n'

In [13]:
# explain the query
test.explain()

'Test: Just get one single arbitrary triple.'

In [14]:
# run the test query with the database connection established above
test.execute(db)

True

In [15]:
# return the results of the executed query
test.results

{'head': {'vars': ['s', 'p', 'o']},
 'results': {'bindings': [{'p': {'type': 'uri',
     'value': 'http://postdata.linhd.uned.es/ontology/postdata-poeticAnalysis#graphName'},
    's': {'type': 'uri',
     'value': 'http://postdata.linhd.uned.es/resource/sc_juana-ines-de-la-cruz_sabras-querido-fabio_plc_16454644863503196'},
    'o': {'type': 'uri',
     'value': 'http://postdata.linhd.uned.es/A_juana-ines-de-la-cruz_sabras-querido-fabio_16454644863503196'}}]}}

In [16]:
test.state

'executed'

### test with a variable in a query

In [17]:
query = """
SELECT * WHERE {
    <$1> ?p ?o .
}
LIMIT 1
"""

In [18]:
test = SparqlQuery(query=query, label="Test with Variable", description="A sample query with a variable.")

In [19]:
test.explain()

'Test with Variable: A sample query with a variable.'

In [20]:
test.dump()

'\nSELECT * WHERE {\n    <$1> ?p ?o .\n}\nLIMIT 1\n'

In [21]:
test.state

'new'

In [22]:
# does it contain variables?
test.query_includes_variables

True

In [23]:
# should not run
#test.execute(db)

In [24]:
# try to inject a value into variable
test.inject(["http://example.uri"])

True

In [25]:
# did it fix?
test.query_includes_variables

False

In [26]:
test.dump()

'\nSELECT * WHERE {\n    <http://example.uri> ?p ?o .\n}\nLIMIT 1\n'

In [27]:
# should be possible to execute
test.execute(db)

True

In [28]:
test.state

'executed'

## Example of a static class template `AuthorsOfPoem`

In [29]:
from pd_stardog_queries import AuthorsOfPoem

In [30]:
imported_test_query = AuthorsOfPoem()

In [31]:
imported_test_query

In [32]:
print(imported_test_query.explain())

Author(s) of a Poem: 
    For a single poem with a "poem_uri" the query returns all URIs of "agents"
    that have the "roleFunction" of "creator" in a relation to a "WorkConception".
    Optionally, it returns a sample name of the author.
    


In [33]:
imported_test_query.template

'\n    SELECT ?Agent (SAMPLE(?Name) AS ?Name)  WHERE {\n        <$1> a pdc:PoeticWork ;\n            pdc:wasInitiatedBy ?WorkConception .\n\n        ?WorkConception pdc:hasAgentRole ?AgentRole .\n\n        ?AgentRole pdc:roleFunction <http://postdata.linhd.uned.es/kos/Creator> ;\n            pdc:hasAgent ?Agent .\n\n        OPTIONAL {\n            ?Agent pdc:name ?Name .\n        }\n    }\n    GROUP BY ?Agent\n    '

In [34]:
# should be prepared but not possible to execute
imported_test_query.state

'prepared'

In [35]:
imported_test_query.prefixes

[{'prefix': 'pdc',
  'uri': 'http://postdata.linhd.uned.es/ontology/postdata-core#'},
 {'prefix': 'pdp',
  'uri': 'http://postdata.linhd.uned.es/ontology/postdata-poeticAnalysis#'},
 {'prefix': 'owl', 'uri': 'http://www.w3.org/2002/07/owl#'}]

In [36]:
# it included the prefixes
imported_test_query.dump()

'PREFIX pdc: <http://postdata.linhd.uned.es/ontology/postdata-core#>\nPREFIX pdp: <http://postdata.linhd.uned.es/ontology/postdata-poeticAnalysis#>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\n    SELECT ?Agent (SAMPLE(?Name) AS ?Name)  WHERE {\n        <$1> a pdc:PoeticWork ;\n            pdc:wasInitiatedBy ?WorkConception .\n\n        ?WorkConception pdc:hasAgentRole ?AgentRole .\n\n        ?AgentRole pdc:roleFunction <http://postdata.linhd.uned.es/kos/Creator> ;\n            pdc:hasAgent ?Agent .\n\n        OPTIONAL {\n            ?Agent pdc:name ?Name .\n        }\n    }\n    GROUP BY ?Agent\n    '

In [37]:
imported_test_query.query_includes_variables

True

In [38]:
imported_test_query.variables

[{'id': 'poem_uri',
  'class': 'pdc:PoeticWork',
  'description': 'URI of a Poem.'}]

In [39]:
test_uri = "http://postdata.linhd.uned.es/resource/pw_juana-ines-de-la-cruz_sabras-querido-fabio"

In [40]:
imported_test_query.inject([test_uri])

True

In [41]:
imported_test_query.query_includes_variables

False

In [42]:
imported_test_query.execute(db)

True

In [43]:
imported_test_query.state

'executed'

In [44]:
imported_test_query.results

{'head': {'vars': ['Agent', 'Name']},
 'results': {'bindings': [{'Agent': {'type': 'uri',
     'value': 'http://postdata.linhd.uned.es/resource/p_juana-ines-de-la-cruz'},
    'Name': {'type': 'literal', 'value': 'Juana Ines de La Cruz'}}]}}

In [45]:
# we would also try the function to initialize with a list of uris from the start
author_poem_query = AuthorsOfPoem(uris=[test_uri])

In [46]:
author_poem_query.execute(db)

True

In [47]:
author_poem_query.results

{'head': {'vars': ['Agent', 'Name']},
 'results': {'bindings': [{'Agent': {'type': 'uri',
     'value': 'http://postdata.linhd.uned.es/resource/p_juana-ines-de-la-cruz'},
    'Name': {'type': 'literal', 'value': 'Juana Ines de La Cruz'}}]}}

In [48]:
# also imeadiatly execute it when initializing:
author_poem_query_execute = AuthorsOfPoem(uris=[test_uri],database=db,execute=True)

In [49]:
author_poem_query_execute.results

{'head': {'vars': ['Agent', 'Name']},
 'results': {'bindings': [{'Agent': {'type': 'uri',
     'value': 'http://postdata.linhd.uned.es/resource/p_juana-ines-de-la-cruz'},
    'Name': {'type': 'literal', 'value': 'Juana Ines de La Cruz'}}]}}

In [50]:
# get a simplified version of the results
author_poem_query_execute.simplified_results()

[{'Agent': 'http://postdata.linhd.uned.es/resource/p_juana-ines-de-la-cruz',
  'Name': 'Juana Ines de La Cruz'}]

In [51]:
mapping = { "Agent" : {"key": "authorUri", "datatype" : "str" }, "Name" : {"key" : "authorName"} }
author_poem_query_execute.simplified_results(mapping=mapping)

[{'authorUri': 'http://postdata.linhd.uned.es/resource/p_juana-ines-de-la-cruz',
  'authorName': 'http://postdata.linhd.uned.es/resource/p_juana-ines-de-la-cruz'}]